In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import urllib
import pandas as pd
import re
import numpy as np

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0'}
session = requests.Session()
link = 'https://www.e*****.**/user/login?current=user/login'
data ={
    "name": "***@gmail.com",
    "pass": "123456",
    "form_build_id": "form-zlVz2TYHRYJInLL_LQtrthethtehA",
    "form_id": "user_login",
    "op": "Log+in"
}
responce = session.post(link, data=data, headers=header)

if responce.status_code != 200:
    print("request denied")
else:
    print("ok")

In [ ]:
def get_html(url, params=None):
    r = session.get(url, headers=header, params=params)
    return r

In [ ]:
# проверка правильности авторизации
url = 'https://www.e*******.**/account'
response = get_html(url)
soup = BeautifulSoup(response.text, 'lxml')
tag_element = soup.find("input", id = "edit-mail")
print(tag_element.get('value'))
assert(tag_element.get('value') == data['name']), 'Autorization Error'

In [ ]:
# проверка правильности выбранной страны
HOST = 'https://www.e*******.**/scholarship'
response = get_html(HOST)
soup = BeautifulSoup(response.text, 'lxml')
tag_element = soup.find("div", id = "overview").p.text.split()
print(tag_element[1] + str(' ') + tag_element[2])

In [ ]:
FQDN = 'https://www.e*******.**/'
HOST = 'https://www.e********.**/scholarship/home'
response = get_html(HOST)
soup = BeautifulSoup(response.text, 'lxml')
rez = soup.tbody

In [ ]:
df = pd.DataFrame()
name = []
url = []
span = []
web_site = []
description = []
requirements = []
deadline=[]

for tag in rez.find_all(['a']):
    if tag.text != ' ':
        name.append(tag.text)
        url.append(FQDN + tag.get('href'))
        
        soup_ext = BeautifulSoup(get_html(url[-1]).text, 'lxml') # страница с детализацией  каждого курса

        # адреса вебсайтов URL
        web_site.append(soup_ext.find("div",
                                       class_='field field-name-field-website field-type-link-field field-label-inline clearfix').find('a').get('href'))
        

        # поля с описанием и требованиями курса. Не везде установлены теги 'p'   
        for e in soup_ext.find_all('div', class_='field-item even'):
            if e.find('h3') and e.find('p'): 
                description.append(e.find_all('p')[0].text)
                requirements.append(e.find_all('p')[1].text)
            elif e.find('h3') and not e.find('p'):
                description.append(e.text.split('\n')[1])
                requirements.append(e.text.split('\n')[3])
        
        # поле с дедлайнами
        for elem in soup_ext.find_all('fieldset', class_='details-deadlines'):
            if re.search('\d+', str(elem)) is not None:
                value = re.sub('[^\d\.]', '', str(elem))
                if 11 < len(value) < 18:
                    deadline.append(value[:5]+', '+value[6:11])
                elif 17 < len(value) < 24:
                    deadline.append(value[:5]+', '+value[6:11]+', '+value[12:17])
                elif 23 < len(value) < 30:
                    deadline.append(value[:5]+', '+value[6:11]+', '+value[12:17]+', '+value[18:23])
                elif len(value) < 7:
                    deadline.append(value[:5])
                else:
                    deadline.append(value)
            else:
                deadline.append(np.nan)
                

for tag in rez.find_all('span'):
    span.append(tag.text)

df['course_name'] = name
df['url'] = url
df['institution'] = span[1::4]
df['amount'] = span[2::4]
df['web_site'] = web_site
df['description'] = description
df['requirements'] = requirements
df['deadline'] = deadline
df

In [ ]:
with pd.ExcelWriter('*******.xlsx') as writer:  
    df.to_excel(writer, sheet_name='UK')